Train TinNet models

In [ ]:
# Loading modules

from __future__ import print_function, division

import numpy as np
import multiprocessing

from ase import io
from ase.db import connect

from tinnet.feature.voronoi import Voronoi
from tinnet.regression.regression import Regression

In [ ]:
# Setting variables

# Optimized hyperparameters
lr = 0.0044485033567158005
atom_fea_len = 106
n_conv = 9
h_fea_len = 60
n_h = 2

db = connect('../Database.db')

d_cen = np.array([r['data']['d_cen'] for r in db.select()], dtype=np.float32)
full_width = np.array([r['data']['full_width'] for r in db.select()], dtype=np.float32)

tabulated_d_cen_inf = np.array([r['data']['tabulated_d_cen_inf'] for r in db.select()], dtype=np.float32)
tabulated_full_width_inf = np.array([r['data']['tabulated_full_width_inf'] for r in db.select()], dtype=np.float32)
tabulated_mulliken = np.array([r['data']['tabulated_mulliken'] for r in db.select()], dtype=np.float32)
tabulated_site_index = np.array([r['data']['tabulated_mulliken'] for r in db.select()], dtype=np.float32)
tabulated_v2dd = np.array([r['data']['tabulated_v2dd'] for r in db.select()], dtype=np.float32)
tabulated_v2ds = np.array([r['data']['tabulated_v2ds'] for r in db.select()], dtype=np.float32)

atom_fea = np.array([r['data']['atom_fea'] for r in db.select()])
nbr_fea = np.array([r['data']['nbr_fea'] for r in db.select()])
nbr_fea_idx = np.array([r['data']['nbr_fea_idx'] for r in db.select()])
tabulated_padding_fillter = np.array([r['data']['tabulated_padding_fillter'] for r in db.select()])

idx = np.arange(len(d_cen))

idx_1 = idx[:-38]
idx_2 = idx[-38:] # Last 38 images are pure metals

num = int(len(idx_1)*1.00) # % of database for training (1.00 means 100%)
np.random.seed(12345)
np.random.shuffle(idx_1)

idx_1 = idx_1[0:num]

idx = np.sort(np.concatenate((idx_1,idx_2)))

d_cen = np.array([d_cen[i] for i in idx])
full_width = np.array([full_width[i] for i in idx])

tabulated_d_cen_inf = np.array([tabulated_d_cen_inf[i] for i in idx])
tabulated_full_width_inf = np.array([tabulated_full_width_inf[i] for i in idx])
tabulated_mulliken = np.array([tabulated_mulliken[i] for i in idx])
tabulated_site_index = np.array([tabulated_site_index[i] for i in idx])
tabulated_v2dd = np.array([tabulated_v2dd[i] for i in idx])
tabulated_v2ds = np.array([tabulated_v2ds[i] for i in idx])

atom_fea = np.array([atom_fea[i] for i in idx], dtype=object)
nbr_fea = np.array([nbr_fea[i] for i in idx], dtype=object)
nbr_fea_idx = np.array([nbr_fea_idx[i] for i in idx], dtype=object)
tabulated_padding_fillter = np.array([tabulated_padding_fillter[i] for i in idx], dtype=object)

final_ans_val_mae = np.zeros(10)
final_ans_val_mse = np.zeros(10)
final_ans_test_mae = np.zeros(10)
final_ans_test_mse = np.zeros(10)

idx_test = 9
idx_validation = 8

In [ ]:
# Setting up the model

model = Regression(atom_fea,
                   nbr_fea,
                   nbr_fea_idx,
                   d_cen,
                   phys_model='moment',
                   optim_algorithm='AdamW',
                   weight_decay=0.0001,
                   idx_validation=idx_validation,
                   idx_test=idx_test,
                   lr=lr,
                   atom_fea_len=atom_fea_len,
                   n_conv=n_conv,
                   h_fea_len=h_fea_len,
                   n_h=n_h,
                   full_width=full_width,
                   tabulated_d_cen_inf=tabulated_d_cen_inf,
                   tabulated_padding_fillter=tabulated_padding_fillter,
                   tabulated_full_width_inf=tabulated_full_width_inf,
                   tabulated_mulliken=tabulated_mulliken,
                   tabulated_site_index=tabulated_site_index,
                   tabulated_v2dd=tabulated_v2dd,
                   tabulated_v2ds=tabulated_v2ds,
                   batch_size=64)

In [ ]:
# Train the network

final_ans_val_mae[idx_test], final_ans_val_mse[idx_test],\
    final_ans_test_mae[idx_test], final_ans_test_mse[idx_test]\
        = model.train(25000)

np.savetxt('final_ans_val_MAE.txt', final_ans_val_mae)
np.savetxt('final_ans_val_MSE.txt', final_ans_val_mse)
np.savetxt('final_ans_test_MAE.txt', final_ans_test_mae)
np.savetxt('final_ans_test_MSE.txt', final_ans_test_mse)